In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [14]:
year = 2022
month =  '04'

In [15]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month}.parquet')

In [16]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [17]:
y_pred.mean()

12.865128336784926

In [9]:
df['ride_id'] = f'{year:04d}/{month}_' + df.index.astype('str')
df_pred = pd.DataFrame(y_pred, columns=['duration'])

df_result = pd.concat([df[['ride_id']], df_pred], axis=1)

df_result.to_parquet(
    "./output_file.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)